# Monolith To MicroServices Udacity Project

In [ ]:
import json
from IPython.utils.text import SList

## Setup Secrets

Copy the [sample.set_env.sh](./sample.set_env.sh) into a new file `set_env.sh`.

Edit the secrets to the appropriate values.

## Setup the AWS Project resources

Set up:

* The RDS (database)
* The S3 bucket (file storage)

Use CloudFormation to make it easier to handle all the related resources together.

Here is the resources dependency graph:

![Resources Dependency Graph](./screenshots/resources-dependency-graph.png)

In [1]:
%%bash
source ./set_env.sh
aws cloudformation create-stack \
  --template-body file://mono-to-micro-template.json \
  --parameters ParameterKey=DBPassword,ParameterValue=$POSTGRES_PASSWORD \
    ParameterKey=DBUsername,ParameterValue=$POSTGRES_USERNAME \
  --stack-name mono-to-micro-project

{
    "StackId": "arn:aws:cloudformation:us-east-1:549763406380:stack/mono-to-micro-project/e1309690-0e6f-11ed-9a25-0a3ba9d153fb"
}


In [28]:
project_stack: SList = !aws cloudformation describe-stacks --stack-name mono-to-micro-project

project_stack

['{',
 '    "Stacks": [',
 '        {',
 '            "StackId": "arn:aws:cloudformation:us-east-1:549763406380:stack/mono-to-micro-project/e1309690-0e6f-11ed-9a25-0a3ba9d153fb",',
 '            "StackName": "mono-to-micro-project",',
 '            "Parameters": [',
 '                {',
 '                    "ParameterKey": "DBPassword",',
 '                    "ParameterValue": "****"',
 '                },',
 '                {',
 '                    "ParameterKey": "DBUsername",',
 '                    "ParameterValue": "****"',
 '                }',
 '            ],',
 '            "CreationTime": "2022-07-28T12:28:14.361000+00:00",',
 '            "RollbackConfiguration": {},',
 '            "StackStatus": "CREATE_COMPLETE",',
 '            "DisableRollback": false,',
 '            "NotificationARNs": [],',
 '            "Outputs": [',
 '                {',
 '                    "OutputKey": "BucketName",',
 '                    "OutputValue": "mono-to-micro-project-s3bucket-1ci

In [29]:
project_stack: dict = json.loads(project_stack.nlstr)
project_stack

{'Stacks': [{'StackId': 'arn:aws:cloudformation:us-east-1:549763406380:stack/mono-to-micro-project/e1309690-0e6f-11ed-9a25-0a3ba9d153fb',
   'StackName': 'mono-to-micro-project',
   'Parameters': [{'ParameterKey': 'DBPassword', 'ParameterValue': '****'},
    {'ParameterKey': 'DBUsername', 'ParameterValue': '****'}],
   'CreationTime': '2022-07-28T12:28:14.361000+00:00',
   'RollbackConfiguration': {},
   'StackStatus': 'CREATE_COMPLETE',
   'DisableRollback': False,
   'NotificationARNs': [],
   'Outputs': [{'OutputKey': 'BucketName',
     'OutputValue': 'mono-to-micro-project-s3bucket-1ciwmj8udutea',
     'Description': 'The S3 Bucket created.'},
    {'OutputKey': 'DBEndpoint',
     'OutputValue': 'mr3tzsiez7j6tp.crxb2aefz7m1.us-east-1.rds.amazonaws.com',
     'Description': 'The database address'},
    {'OutputKey': 'DBPort',
     'OutputValue': '5432',
     'Description': 'The database port'}],
   'Tags': [],
   'EnableTerminationProtection': False,
   'DriftInformation': {'StackDri

In [33]:
bucket_name, db_host, db_port = None, None, None
for output in project_stack['Stacks'][0]['Outputs']:
  if output['OutputKey'] == 'BucketName':
    bucket_name = output['OutputValue']
  elif output['OutputKey'] == 'DBEndpoint':
    db_host = output['OutputValue']
  elif output['OutputKey'] == 'DBPort':
    db_port = output['OutputValue']
    
bucket_name, db_host, db_port

('mono-to-micro-project-s3bucket-1ciwmj8udutea',
 'mr3tzsiez7j6tp.crxb2aefz7m1.us-east-1.rds.amazonaws.com',
 '5432')

## Clean up used resources

### Empty S3 Bucket

In [4]:
%%bash
aws s3 rm s3://{bucket_name} \
  --recursive

delete: s3://mono-to-micro-project-s3bucket-150yzcsnu6pp0/xander0.jpg
delete: s3://mono-to-micro-project-s3bucket-150yzcsnu6pp0/xander2.jpg


### Delete the CloudFormation Stack

In [5]:
%%bash
aws cloudformation delete-stack \
  --stack-name mono-to-micro-project